In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.loaders import LandsatOdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter

from src.run_task import GeoMADLandsatProcessor, get_grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Issues
# region_code = "66,19"  # BBOX is bad, baby

# Study site configuration
# region_code = "68,18"
datetime = "2023"
item_id = region_code

# And get the study site
grid = get_grid()
cell = grid.loc[[(region_code)]]
cell.explore()

In [ ]:
# Set up a data loader
loader = LandsatOdcLoader(
    epsg=3832,
    datetime=datetime,
    dask_chunksize=dict(time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False,
        resolution=30,
        # bands=["qa_pixel", "red", "green", "blue", "nir08", "swir16", "swir22"],
        bands=["qa_pixel", "red", "green", "blue"]
    ),
    exclude_platforms=["landsat-7"],
    nodata_value=0,
    keep_ints=True,
    flat_array=True,
    only_tier_one=True,
    fall_back_to_tier_two=True
)

# Run the load process, which is lazy-loaded
input_data = loader.load(cell)
input_data

In [ ]:
from dep_tools.utils import search_across_180

bbox = cell.to_crs(4326).total_bounds
if bbox[0] > 180:
    bbox[0] = bbox[0] - 360
    if bbox[2] > 180:
        bbox[2] = bbox[2] - 360

bbox_crosses_antimeridian = (
    bbox[0] < 0 and bbox[2] > 0 or bbox[0] < 180 and bbox[2] > 180
)
if bbox_crosses_antimeridian:
    print("it crosses")
    xmin_ll, ymin_ll = bbox[0], bbox[1]
    xmax_ll, ymax_ll = bbox[2], bbox[3]

    xmax_ll = xmax_ll - 360 if xmax_ll > 180 else xmax_ll

    left_bbox = [xmin_ll, ymin_ll, 180, ymax_ll]
    right_bbox = [-180, ymin_ll, xmax_ll, ymax_ll]

In [ ]:
# Set up a data processor
processor = GeoMADLandsatProcessor(
    scale_and_offset=False,
    dilate_mask=[2, 3],
    work_chunks=(601, 601),
    num_threads=10,
    keep_ints=True
)

# Plan the processing. Still lazy-loaded
output_data = processor.process(input_data)
output_data

In [ ]:
# Actually load data and do the processing, so we have it in memory
loaded = output_data.compute()
loaded

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
data.red.isel(time=0).plot.imshow(size=8, robust=True)

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="ls",
    dataset_id="geomad",
    version="0.0.0",
    time=datetime,
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
out_files = writer.write(loaded, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()